# CNN Model for Kinect data

In [1]:
import os 
import tempfile

import numpy as np 
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.layers import MaxPool1D

# MLflow dashboard
import mlflow
mlflow.set_tracking_uri('http://35.228.45.76:5000')
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='../../mlflow-312506-8cfad529f4fd.json'

# Import data augmentation
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [4]:
random_state = 47
np.random.seed(random_state)

In [5]:
from keras.callbacks import EarlyStopping
#monitor='val_accuracy',
#monitor='val_prc', 
early_stopping = EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

## 1. Load Prepared Dataset

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/digitacs/4dv652-ml/main/datasets/all_good_bad_problemA_kinect/good_bad_kinect.csv')

df.fillna(0, inplace=True) # Replace NaN with 0

df.shape

(111416, 40)

## 2. Split into sets for training, validation, and testing + use and save scaler

In [7]:
X = df.drop(columns=['quality'])
y = df['quality']

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=random_state)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print('Training features shape:', X_train.shape)
print('Training labels shape:', y_train.shape, '\n')

print('Validation features shape:', X_val.shape)
print('Validation labels shape:', y_val.shape, '\n')

print('Test features shape:', X_test.shape)
print('Test labels shape:', y_test.shape, '\n')

Training features shape: (80219, 39)
Training labels shape: (80219,) 

Validation features shape: (20055, 39)
Validation labels shape: (20055,) 

Test features shape: (11142, 39)
Test labels shape: (11142,) 



## 3. Define the Model

### 3.1 Configurations

In [10]:
from numpy import unique
orig_shape = X_train.shape[1:]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

in_shape = X_train.shape[1:]
print("Before: {0}".format(orig_shape))
print("After: {0}".format(in_shape))



EPOCHS = 500
BATCH_SIZE = 2050

n_classes = len(unique(y_train))
n_classes = 1
print("Classes: {0}".format(n_classes))

units = 64
activation = 'relu'
kernel_initializer = 'he_uniform'
output_activation = 'sigmoid'
#output_activation = 'softmax'
optimizer = 'Adam'
learning_rate = 1e-3
filters = 1
kernel_size = 1

Before: (39,)
After: (39, 1)
Classes: 1


### 3.2 Model Architecture

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer

from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Dropout, Flatten
from keras.layers import BatchNormalization

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(loss, optimizer, learning_rate=0.001, metrics=METRICS):
    
    model = Sequential()
    model.add(Conv1D(filters= filters, kernel_size = kernel_size, activation=activation, kernel_initializer=kernel_initializer, input_shape=in_shape))
    #model.add(BatchNormalization())
    #model.add(MaxPool1D(1))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
    #model.add(Dense(8, activation='relu', kernel_initializer='he_uniform'))
    #model.add(Dropout(0.5))
    #model.add(BatchNormalization())
    model.add(Dense(n_classes, activation=output_activation))
    
    
    optimizer = tf.keras.optimizers.get(optimizer)
    optimizer.learning_rate.assign(learning_rate)

    model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=metrics)

    print(model.summary())

    return model

## 4. MLflow Experimentation

### 4.1 Helper functions

In [12]:
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve

def plot_loss(history, label, n):
  plt.semilogy(history.epoch, history.history['loss'],
               color=colors[n], label='Train ' + label)
  plt.semilogy(history.epoch, history.history['val_loss'],
               color=colors[n], label='Val ' + label,
               linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')

def plot_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    print('True Negatives: ', cm[0][0])
    print('False Positives: ', cm[0][1])
    print('False Negatives: ', cm[1][0])
    print('True Positives: ', cm[1][1])
    print('Total: ', np.sum(cm[1]))


def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

def plot_prc(name, labels, predictions, **kwargs):
    precision, recall, _ = precision_recall_curve(labels, predictions)

    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

### 4.2 Start Run

In [ ]:
model_name = 'cnn_model_kinect'
#loss='sparse_categorical_crossentropy'
#loss='categorical_crossentropy'
#loss='binary_crossentropy'

with mlflow.start_run(run_name=model_name) as run:

    run_id = run.info.run_id

    model = make_model(metrics=METRICS, 
                        loss='binary_crossentropy', 
                        optimizer=optimizer, 
                        learning_rate=learning_rate)
    
    with tf.device('/CPU:0'):
        
        history = model.fit(
            x=X_train, 
            y=y_train, 
            validation_data=(X_val, y_val), 
            shuffle=True, 
            epochs=EPOCHS, 
            verbose=1,
            batch_size=BATCH_SIZE, 
            callbacks=[early_stopping])

        # Plot training history
        plot_loss(history, "Bias", 0)
        plt.savefig("loss.jpg")
        #mlflow.log_artifact("loss.jpg")
        plt.show()
        
        # Plot and log metrics
        predictions_train = model.predict(X_train)
        predictions_test = model.predict(X_test)

        plot_metrics(history)
        plt.savefig("metrics.jpg")
        #mlflow.log_artifact("metrics.jpg")
        plt.show()

        # Confusion matrix
        results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
        for name, value in zip(model.metrics_names, results):
            print(name, ': ', value, '/n')
        plot_cm(y_test, predictions_test)
        plt.savefig("cm.jpg")
        #mlflow.log_artifact("cm.jpg")
        plt.show()

        # ROC Curve
        plot_roc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
        plt.legend(loc='lower right')
        plt.savefig("roc.jpg")
        #mlflow.log_artifact("roc.jpg")
        plt.show()

        # Precision-Recall Curve (PRC)
        plot_prc("Train Predictions", y_train, predictions_train, color=colors[0])
        plot_prc("Test Predictions", y_test, predictions_test, color=colors[0], linestyle='--')
        plt.legend(loc='lower right')
        plt.savefig("prc.jpg")
        #mlflow.log_artifact("prc.jpg")
        plt.show()
        
        # Log model, scaler, model parameters to MLflow
        #mlflow.log_param("units", units)
        #mlflow.log_param("activation", activation)
        #mlflow.log_param("kernel_initializer", kernel_initializer)
        #mlflow.log_param("output activation", output_activation)
        #mlflow.log_param("optimizer", optimizer)
        #mlflow.log_param("learning rate", learning_rate)
        #mlflow.log_param("batch size", BATCH_SIZE)
        #mlflow.log_metric("accuracy", results[5])
        #mlflow.log_metric("average precision score", results[6])
        #mlflow.log_metric("recall", results[7])
        #mlflow.log_metric("auc", results[8])

        #mlflow.keras.log_model(model, model_name, signature=signature)
        #mlflow.sklearn.log_model(scaler, 'InputScaler')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 39, 1)             2         
_________________________________________________________________
flatten_1 (Flatten)          (None, 39)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1280      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 1,827
Trainable params: 1,827
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
40/40 [==============================] - 2s 25ms/step - loss: 0.5819 - tp: 46496.2683 - fp: 9513.5854

## 5. Register Model

In [ ]:
model_uri = "runs:/{}/{}".format(run.info.run_id, model_name)
mv = mlflow.register_model(model_uri, model_name)
print("Name: {}".format(mv.name))
print("Version: {}".format(mv.version))